In [9]:
import pandas as pd
import hopsworks
import CleanInputFunction
import TopicExtractionFunction

In [10]:
# Connect to Hopsworks
project = hopsworks.login()
feature_store = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5270
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
# Read in training data
data = pd.read_csv('data/RoundedLabelled.csv')
data = data.rename(columns={'title': 'title', 'topic': 'subjects', 'stance / attitude towards': 'stance', 'article URL': 'url', 'savedAt': 'saved_at', 'main topic': 'labelled_topic'})
data = data.drop(['query', 'language', 'Clickbaityness/credibility (0 is good, 1 is very bad)', 'labelled_topic'], axis=1) # labelled_topic is dropped as this is only for testing the topic extraction and is not used as training data, Hopsworks requires all columns to be filled which won't always be the case for labelled_topic
data = data[data['stance'].notnull()]
data['stance'] = data['stance'].astype('double')
data

,title,saved_at,url,stance
0,Fed interest rate decision today: Live updates...,14/12/2022,https://www.usatoday.com/story/money/economy/2...,2.0
1,Moscow police should hand over control of Idah...,14/12/2022,https://nypost.com/2022/12/14/police-should-ha...,2.0
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022,https://www.bbc.com/news/world-africa-63973938,2.0
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022,https://www.youtube.com/watch?v=PesQLjkAEew,2.0
4,Mortgage demand inches higher as interest rate...,14/12/2022,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0
...,...,...,...,...
487,Russia's war in Ukraine: Live updates - CNN,27/12/2022,https://www.cnn.com/europe/live-news/russia-uk...,2.0
488,Russia’s Lavrov issues ultimatum to Ukraine: ‘...,27/12/2022,https://www.aljazeera.com/news/2022/12/27/russ...,2.0
489,Southwest Airlines apparently cancels all depa...,27/12/2022,https://ktla.com/news/local-news/southwest-can...,1.0
490,The 7 most spectacular images from the James W...,27/12/2022,https://www.salon.com/2022/12/25/the-7-most-sp...,3.0


In [12]:
# Pre-process
data = CleanInputFunction.clean_input(data)
display(data)

,title,saved_at,url,stance,title_stance,title_topic
0,Fed interest rate decision today: Live updates...,14/12/2022,https://www.usatoday.com/story/money/economy/2...,2.0,fed interest rate decision today live updates ...,fed interest rate decision today live updates ...
1,Moscow police should hand over control of Idah...,14/12/2022,https://nypost.com/2022/12/14/police-should-ha...,2.0,moscow police should hand over control of idah...,moscow police hand control idaho murder invest...
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022,https://www.bbc.com/news/world-africa-63973938,2.0,world cup 2022 morocco fans heartbreak as doha...,world cup 2022 morocco fans heartbreak doha fl...
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022,https://www.youtube.com/watch?v=PesQLjkAEew,2.0,scream 6 official teaser trailer 2023 jenna or...,scream 6 official teaser trailer 2023 jenna or...
4,Mortgage demand inches higher as interest rate...,14/12/2022,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0,mortgage demand inches higher as interest rate...,mortgage demand inches higher interest rates m...
...,...,...,...,...,...,...
487,Russia's war in Ukraine: Live updates - CNN,27/12/2022,https://www.cnn.com/europe/live-news/russia-uk...,2.0,russia s war in ukraine live updates cnn,russia war ukraine live updates cnn
488,Russia’s Lavrov issues ultimatum to Ukraine: ‘...,27/12/2022,https://www.aljazeera.com/news/2022/12/27/russ...,2.0,russia s lavrov issues ultimatum to ukraine fo...,russia lavrov issues ultimatum ukraine good al...
489,Southwest Airlines apparently cancels all depa...,27/12/2022,https://ktla.com/news/local-news/southwest-can...,1.0,southwest airlines apparently cancels all depa...,southwest airlines apparently cancels departin...
490,The 7 most spectacular images from the James W...,27/12/2022,https://www.salon.com/2022/12/25/the-7-most-sp...,3.0,the 7 most spectacular images from the james w...,7 spectacular images james webb space telescop...


In [13]:
# Add topics
data, _ = TopicExtractionFunction.extract_topics(data)
display(data)

2022-12-29 11:31:57,537 INFO: Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
2022-12-29 11:31:59,387 INFO: Use pytorch device: cpu


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-12-29 11:32:06,011 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2022-12-29 11:32:06,149 INFO: Use pytorch device: cpu


,title,saved_at,url,stance,title_stance,title_topic,predicted_topic
0,Fed interest rate decision today: Live updates...,14/12/2022,https://www.usatoday.com/story/money/economy/2...,2.0,fed interest rate decision today live updates ...,fed interest rate decision today live updates ...,"inflation, fed, rates"
1,Moscow police should hand over control of Idah...,14/12/2022,https://nypost.com/2022/12/14/police-should-ha...,2.0,moscow police should hand over control of idah...,moscow police hand control idaho murder invest...,"shooting, paris, kurds"
2,World Cup 2022: Morocco fans' heartbreak as Do...,14/12/2022,https://www.bbc.com/news/world-africa-63973938,2.0,world cup 2022 morocco fans heartbreak as doha...,world cup 2022 morocco fans heartbreak doha fl...,"cup, africa, world"
3,Scream 6 - Official Teaser Trailer (2023) Jenn...,14/12/2022,https://www.youtube.com/watch?v=PesQLjkAEew,2.0,scream 6 official teaser trailer 2023 jenna or...,scream 6 official teaser trailer 2023 jenna or...,"weekend, showdown, avatar"
4,Mortgage demand inches higher as interest rate...,14/12/2022,https://www.cnbc.com/2022/12/14/mortgage-deman...,2.0,mortgage demand inches higher as interest rate...,mortgage demand inches higher interest rates m...,"mortgage, rates, television"
...,...,...,...,...,...,...,...
487,Russia's war in Ukraine: Live updates - CNN,27/12/2022,https://www.cnn.com/europe/live-news/russia-uk...,2.0,russia s war in ukraine live updates cnn,russia war ukraine live updates cnn,"ukraine, russia, war"
488,Russia’s Lavrov issues ultimatum to Ukraine: ‘...,27/12/2022,https://www.aljazeera.com/news/2022/12/27/russ...,2.0,russia s lavrov issues ultimatum to ukraine fo...,russia lavrov issues ultimatum ukraine good al...,"ukraine, russia, war"
489,Southwest Airlines apparently cancels all depa...,27/12/2022,https://ktla.com/news/local-news/southwest-can...,1.0,southwest airlines apparently cancels all depa...,southwest airlines apparently cancels departin...,"flights, airlines, departing"
490,The 7 most spectacular images from the James W...,27/12/2022,https://www.salon.com/2022/12/25/the-7-most-sp...,3.0,the 7 most spectacular images from the james w...,7 spectacular images james webb space telescop...,"nasa, webb, mars"


In [14]:
# Save result to feature_store
article_cleaned_feature_store = feature_store.get_or_create_feature_group(
    name="training_data_stance",
    version=1,
    primary_key=["url"],
    description="Articles with predicted topic")
article_cleaned_feature_store.insert(data, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/492 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5270/jobs/named/training_data_stance_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x18a576dfcd0>, None)